# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0098c03d30>
├── 'a' --> tensor([[ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687]])
└── 'x' --> <FastTreeValue 0x7f0098c03e50>
    └── 'c' --> tensor([[-1.0120,  0.6890, -1.2714,  0.3398],
                        [-0.7247, -1.2218,  0.3855, -1.5048],
                        [-0.9781, -1.6866, -0.5531, -1.0940]])

In [4]:
t.a

tensor([[ 0.2339, -0.1569, -0.7355],
        [ 1.2830, -0.0808, -1.2687]])

In [5]:
%timeit t.a

49.2 ns ± 0.504 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0098c03d30>
├── 'a' --> tensor([[ 0.4779, -0.1263, -1.3072],
│                   [-0.3864,  0.7463,  0.3715]])
└── 'x' --> <FastTreeValue 0x7f0098c03e50>
    └── 'c' --> tensor([[-1.0120,  0.6890, -1.2714,  0.3398],
                        [-0.7247, -1.2218,  0.3855, -1.5048],
                        [-0.9781, -1.6866, -0.5531, -1.0940]])

In [7]:
%timeit t.a = new_value

50.7 ns ± 0.0748 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687]]),
    x: Batch(
           c: tensor([[-1.0120,  0.6890, -1.2714,  0.3398],
                      [-0.7247, -1.2218,  0.3855, -1.5048],
                      [-0.9781, -1.6866, -0.5531, -1.0940]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2339, -0.1569, -0.7355],
        [ 1.2830, -0.0808, -1.2687]])

In [11]:
%timeit b.a

40.9 ns ± 0.263 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.3346, -1.1481, -0.6253],
               [-0.2721,  0.2483, -0.1989]]),
    x: Batch(
           c: tensor([[-1.0120,  0.6890, -1.2714,  0.3398],
                      [-0.7247, -1.2218,  0.3855, -1.5048],
                      [-0.9781, -1.6866, -0.5531, -1.0940]]),
       ),
)

In [13]:
%timeit b.a = new_value

359 ns ± 0.511 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

697 ns ± 6.39 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

8.59 µs ± 30.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

128 µs ± 528 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

128 µs ± 514 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7effbbf1fb20>
├── 'a' --> tensor([[[ 0.2339, -0.1569, -0.7355],
│                    [ 1.2830, -0.0808, -1.2687]],
│           
│                   [[ 0.2339, -0.1569, -0.7355],
│                    [ 1.2830, -0.0808, -1.2687]],
│           
│                   [[ 0.2339, -0.1569, -0.7355],
│                    [ 1.2830, -0.0808, -1.2687]],
│           
│                   [[ 0.2339, -0.1569, -0.7355],
│                    [ 1.2830, -0.0808, -1.2687]],
│           
│                   [[ 0.2339, -0.1569, -0.7355],
│                    [ 1.2830, -0.0808, -1.2687]],
│           
│                   [[ 0.2339, -0.1569, -0.7355],
│                    [ 1.2830, -0.0808, -1.2687]],
│           
│                   [[ 0.2339, -0.1569, -0.7355],
│                    [ 1.2830, -0.0808, -1.2687]],
│           
│                   [[ 0.2339, -0.1569, -0.7355],
│                    [ 1.2830, -0.0808, -1.2687]]])
└── 'x' --> <FastTreeValue 0x7effbbf1f400>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

23.8 µs ± 37 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7effb3cd7610>
├── 'a' --> tensor([[ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687],
│                   [ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687],
│                   [ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687],
│                   [ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687],
│                   [ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687],
│                   [ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687],
│                   [ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687],
│                   [ 0.2339, -0.1569, -0.7355],
│                   [ 1.2830, -0.0808, -1.2687]])
└── 'x' --> <FastTreeValue 0x7effb3cd7430>
    └── 'c' --> tensor([[-1.0120,  0.6890, -1.2714,  0.3398],
                        [-0.7247, -1.2218,  0.3855, -1.5048],
                 

In [23]:
%timeit t_cat(trees)

22.3 µs ± 77.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

50.3 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2339, -0.1569, -0.7355],
                [ 1.2830, -0.0808, -1.2687]],
       
               [[ 0.2339, -0.1569, -0.7355],
                [ 1.2830, -0.0808, -1.2687]],
       
               [[ 0.2339, -0.1569, -0.7355],
                [ 1.2830, -0.0808, -1.2687]],
       
               [[ 0.2339, -0.1569, -0.7355],
                [ 1.2830, -0.0808, -1.2687]],
       
               [[ 0.2339, -0.1569, -0.7355],
                [ 1.2830, -0.0808, -1.2687]],
       
               [[ 0.2339, -0.1569, -0.7355],
                [ 1.2830, -0.0808, -1.2687]],
       
               [[ 0.2339, -0.1569, -0.7355],
                [ 1.2830, -0.0808, -1.2687]],
       
               [[ 0.2339, -0.1569, -0.7355],
                [ 1.2830, -0.0808, -1.2687]]]),
    x: Batch(
           c: tensor([[[-1.0120,  0.6890, -1.2714,  0.3398],
                       [-0.7247, -1.2218,  0.3855, -1.5048],
                       [-0.9781, -1.6866, -0.5531, -1.0940]],
         

In [26]:
%timeit Batch.stack(batches)

63 µs ± 445 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687],
               [ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687],
               [ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687],
               [ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687],
               [ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687],
               [ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687],
               [ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687],
               [ 0.2339, -0.1569, -0.7355],
               [ 1.2830, -0.0808, -1.2687]]),
    x: Batch(
           c: tensor([[-1.0120,  0.6890, -1.2714,  0.3398],
                      [-0.7247, -1.2218,  0.3855, -1.5048],
                      [-0.9781, -1.6866, -0.5531, -1.0940],
                      [-1.0120,  0.6890, -1.2714,  0.3398],
                      [-0.7247, -

In [28]:
%timeit Batch.cat(batches)

119 µs ± 543 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

284 µs ± 4.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
